In [1]:
%load_ext autoreload
%autoreload 2

## Generating summaries

In [2]:
import os
while 'notebooks' in os.getcwd():
    os.chdir("..")

import numpy as np
import pandas as pd 
from src.utils import train_test_split, get_sample_weights, get_eval_set
from src.preprocessing import preprocess_data
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel, AutoModelForCausalLM
from src.preprocessing import TextDataset
import torch
from torch.utils.data import DataLoader, Dataset
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, roc_auc_score
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, LoggingHandler
import logging
from copy import deepcopy
from sklearn.decomposition import PCA
from huggingface_hub import notebook_login
from sklearn.ensemble import RandomForestClassifier
from collections import defaultdict
import transformers
import re
from src.utils import aggregate_samples, evaluate_model, compute_class_weights, remove_hashtag_links, get_first_texts
from torch.optim.lr_scheduler import ReduceLROnPlateau
import ast

import asyncio
from openai import OpenAI

tqdm.pandas()

import nest_asyncio
nest_asyncio.apply()

In [6]:
train_data_processed, _ = train_test_split(preprocess=True)
train_data_raw, _ = train_test_split(preprocess=False)


100%|██████████| 16/16 [00:05<00:00,  3.03it/s]


In [14]:
df_1 = pd.concat(train_data_raw)
df_2 = pd.concat(train_data_processed)


In [15]:
num_total_tweets = df_1.groupby(["MatchID", "PeriodID"])\
    [["ID"]]\
    .count()\
    .rename(columns={"ID": "num_total_tweets"})

num_total_tweets_match = df_1.groupby(["MatchID"])\
    [["ID"]]\
    .count()\
    .rename(columns={"ID": "num_tweets_match"})

In [16]:
num_processed_tweets = df_2.groupby(["MatchID", "PeriodID"])\
    [["ID"]]\
    .count()\
    .rename(columns={"ID": "num_processed_tweets"})

num_processed_tweets_match = df_2.groupby(["MatchID"])\
    [["ID"]]\
    .count()\
    .rename(columns={"ID": "num_tweets_match_processed"})

In [18]:
labels = df_1.groupby(["MatchID", "PeriodID"])\
    [["EventType"]]\
    .first()\
    .rename(columns={"ID": "label"})

In [22]:
features = pd.concat([num_processed_tweets, num_total_tweets, labels], axis = 1)

features_match = pd.concat([num_processed_tweets_match, num_total_tweets_match], axis=1)

In [33]:
X_train = pd.merge(
    features,
    features_match,
    on = "MatchID",
)

In [35]:
X_train.index=  features.index

In [55]:
match_mask = X_train.index.get_level_values("MatchID") == 1
X_match = X_train[match_mask]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [64]:
X1 = X_train.query("MatchID in [0,1,2,3,4]").astype(float)
X2 = X_train.query("MatchID not in [0,1,2,3,4]").astype(float)

In [86]:
model = RandomForestClassifier(max_depth=3)

model.fit(X1[X1.columns.difference(["EventType"])], X1['EventType'])

RandomForestClassifier(max_depth=3)

In [87]:
y_pred = model.predict(X2[X1.columns.difference(["EventType"])])

In [88]:
accuracy_score(y_pred, X2['EventType'])

0.630960608154803

In [49]:
X_match

num_processed_tweets  num_total_tweets  EventType  \
MatchID PeriodID                                                      
1       0                         1710              5747          0   
        1                         1876              5332          0   
        2                         2013              5019          0   
        3                         2264              5154          0   
        4                         2305              5101          1   
...                                ...               ...        ...   
        125                       1642              7545          1   
        126                       1578              7425          0   
        127                       1451              7640          1   
        128                       1436              7667          0   
        129                       1390              7781          0   

                  num_tweets_match_processed  num_tweets_match  
MatchID PeriodID                                                
1       0                             286978            973985  
        1                             286978            973985  
        2                             286978            973985  
        3                             286978            973985  
        4                             286978            973985  
...                                      ...               ...  
        125                           286978            973985  
        126                           286978            973985  
        127                           286978            973985  
        128                           286978            973985  
        129                           286978            973985  

[130 rows x 5 columns]